In [3]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!pip install pandas

In [4]:
import pandas as pd

# Specify the file path in Google Drive
file_path = '/content/drive/MyDrive/sales.json'

# Read the JSON file into a Pandas DataFrame
salesdf = pd.read_json(file_path, lines=True)


In [6]:
num_rows, num_columns = salesdf.shape

print(f"Number of Rows: {num_rows}")
print(f"Number of Columns: {num_columns}")

Number of Rows: 5000
Number of Columns: 7
